In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [48]:
data = pd.read_csv('/content/heart_.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [49]:
le_model = LabelEncoder()

data['Sex'] = le_model.fit_transform(data['Sex'])
data['ChestPainType'] = le_model.fit_transform(data['ChestPainType'])
data['RestingECG'] = le_model.fit_transform(data['RestingECG'])
data['ExerciseAngina'] = le_model.fit_transform(data['ExerciseAngina'])
data['ST_Slope'] = le_model.fit_transform(data['ST_Slope'])

In [50]:
# Set random seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

In [51]:
X=data.drop('HeartDisease', axis=1)

y=data['HeartDisease']

In [52]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [53]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
#  ANN model with fixed seeds
ANN = keras.Sequential([
    layers.Input(shape=(11,)),
    layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),  
    layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),  
    layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
])

In [55]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [56]:
# Train the model with early stopping
ANN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Make predictions
y_pred = ANN.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary values

# Calculate evaluation metrics
accuracy_ANN = accuracy_score(y_test, y_pred)
precision_ANN = precision_score(y_test, y_pred)
recall_ANN = recall_score(y_test, y_pred)
f1_ANN = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import roc_auc_score


y_pred_prob_ANN = ANN.predict(X_test).ravel()
auc_roc_ANN = roc_auc_score(y_test, y_pred_prob_ANN)
print(f'ANN AUC: {auc_roc_ANN}')


print(f'Accuracy: {accuracy_ANN:.2f}')
print(f'Precision: {precision_ANN:.2f}')
print(f'Recall: {recall_ANN:.2f}')
print(f'F1 Score: {f1_ANN:.2f}')
print('Confusion Matrix:')
print(confusion)

Epoch 1/100
24/24 [==============================] - 1s 10ms/step - loss: 0.6184 - accuracy: 0.6662 - val_loss: 0.4510 - val_accuracy: 0.9157
Epoch 2/100
24/24 [==============================] - 0s 4ms/step - loss: 0.5405 - accuracy: 0.7376 - val_loss: 0.3900 - val_accuracy: 0.9157
Epoch 3/100
24/24 [==============================] - 0s 4ms/step - loss: 0.5185 - accuracy: 0.7443 - val_loss: 0.3512 - val_accuracy: 0.8795
Epoch 4/100
24/24 [==============================] - 0s 4ms/step - loss: 0.4758 - accuracy: 0.7860 - val_loss: 0.3282 - val_accuracy: 0.8795
Epoch 5/100
24/24 [==============================] - 0s 3ms/step - loss: 0.4541 - accuracy: 0.8075 - val_loss: 0.3113 - val_accuracy: 0.8795
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 0.4192 - accuracy: 0.8143 - val_loss: 0.3037 - val_accuracy: 0.8795
Epoch 7/100
24/24 [==============================] - 0s 4ms/step - loss: 0.4248 - accuracy: 0.8237 - val_loss: 0.3046 - val_accuracy: 0.8916
Epoch 8/100


Epoch 59/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3288 - accuracy: 0.8560 - val_loss: 0.2307 - val_accuracy: 0.8916
Epoch 60/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3200 - accuracy: 0.8816 - val_loss: 0.2306 - val_accuracy: 0.8916
Epoch 61/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3280 - accuracy: 0.8694 - val_loss: 0.2375 - val_accuracy: 0.8916
Epoch 62/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3246 - accuracy: 0.8600 - val_loss: 0.2375 - val_accuracy: 0.8916
Epoch 63/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3127 - accuracy: 0.8735 - val_loss: 0.2347 - val_accuracy: 0.8916
Epoch 64/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3256 - accuracy: 0.8641 - val_loss: 0.2292 - val_accuracy: 0.8916
Epoch 65/100
24/24 [==============================] - 0s 4ms/step - loss: 0.3222 - accuracy: 0.8762 - val_loss: 0.2313 - val_accuracy: 0.8916
Epoch 